# Pulling Venue Data from FourSquare API

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Testing the Code](#Testing-the-Code)

[Making the DataFrame](#Making-the-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Venue and Distance](#Sorting-by-Venue-and-Distance)

[Obtaining Neighborhood for Each Arcade](#Obtaining-Neighborhood-for-Each-Arcade)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [3]:
kc_df = pd.read_csv('Kansas_City_geodata.csv')
kc_df.head(10)

,Area,Neighborhood,area_longitude,area_latitude,hood_longitude,hood_latitude
0,CBD-Downtown,CBD-Downtown,-94.5787868,39.0917491,-94.5787868,39.0917491
1,Greater Downtown,18th and Vine,No Data Found,No Data Found,-94.55388639189565,39.09444044999999
2,Greater Downtown,Beacon Hill-McFeders,No Data Found,No Data Found,No Data Found,No Data Found
3,Greater Downtown,Columbus Park,No Data Found,No Data Found,-94.5703817,39.110399
4,Greater Downtown,Crossroads,No Data Found,No Data Found,-94.5837436,39.0901009
5,Greater Downtown,Hospital Hill,No Data Found,No Data Found,-94.5753134,39.0844079
6,Greater Downtown,Library District,No Data Found,No Data Found,-94.5829967,39.1032931
7,Greater Downtown,Longfellow/Dutch Hill,No Data Found,No Data Found,No Data Found,No Data Found
8,Greater Downtown,Quality Hill,No Data Found,No Data Found,-94.590331,39.1018009
9,Greater Downtown,River Market,No Data Found,No Data Found,-94.58115944495933,39.11323435


### Connecting to FourSquare

In [4]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [5]:
VERSION = '20180604'
LIMIT = 30

### Testing the Code

In [6]:
#assign variables for the search
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [7]:
for index, hood in enumerate(kc_df['hood_latitude']):
    if index == 6: #preselected based on venues in initial search
        latitude= kc_df['hood_latitude'][index]
        longitude= kc_df['hood_longitude'][index]
print(latitude, longitude)

39.1032931 -94.5829967


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6142175089484f35270a7127'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '58864fb051d19e79a1fdcf0e',
    'name': 'Tavernonna',
    'location': {'address': '106 W 12th St',
     'crossStreet': 'at Baltimore Ave',
     'lat': 39.10008108845158,
     'lng': -94.58454886613707,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.10008108845158,
       'lng': -94.58454886613707}],
     'distance': 381,
     'postalCode': '64105',
     'cc': 'US',
     'city': 'Kansas City',
     'state': 'MO',
     'country': 'United States',
     'formattedAddress': ['106 W 12th St (at Baltimore Ave)',
      'Kansas City, MO 64105']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png

In [10]:
ident=results['response']['venues'][0]
type(ident)
for dictionary in results['response']['venues']:
    print(dictionary['id'])
    print(dictionary['name'])
    print(dictionary['location']['address'])
    print(dictionary['location']['lat'])
    print(dictionary['location']['lng'])
    print(dictionary['location']['distance'])
    for cat in dictionary['categories']:
        print(cat['id'], cat['name'])

58864fb051d19e79a1fdcf0e
Tavernonna
106 W 12th St
39.10008108845158
-94.58454886613707
381
4bf58dd8d48988d110941735 Italian Restaurant


### Making the DataFrame

In [11]:
#assign variables for the search
search_query = 'Arcade'
radius = 2000
print(search_query + ' .... OK!')

Arcade .... OK!


In [12]:
kc_venues = pd.DataFrame(columns = ['Hood','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
kc_venues.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [15]:
for index, hood in enumerate(kc_df['hood_latitude']):
    try:
        hood_name = kc_df['Neighborhood'][index]
        latitude = kc_df['hood_latitude'][index]
        longitude = kc_df['hood_longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        continue
    try:
        res = results['response']['venues']
    except:
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
                continue
        kc_venues = kc_venues.append({'Venue_Distance':ven_dist,'Hood':hood_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)

In [16]:
kc_venues = kc_venues.set_index('Hood')
print(len(kc_venues))
kc_venues.head()

73


,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
Hood,,,,,,,,
CBD-Downtown,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1130,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
CBD-Downtown,54076348498ed3ee81d32983,Scarritt Arcade,1361,819 Walnut St,39.103793,-94.581555,4bf58dd8d48988d130941735,Building
CBD-Downtown,5ad817f3c666667613ddcb70,Broadway Arcade,1449,1000 Broadway Blvd Ste 200,39.102272,-94.588670,4bf58dd8d48988d1e1931735,Arcade
CBD-Downtown,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1120,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
Columbus Park,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1437,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza


In [17]:
print(len(kc_venues))
kc_venues.to_csv('{}_venue_data.csv'.format(search_query))

73


### Dropping the Nulls

In [19]:
kc_venues = pd.read_csv('{}_venue_data.csv'.format(search_query))
print(search_query)
print(len(kc_venues))
kc_venues.head()

Arcade
73


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,CBD-Downtown,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1130,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
1,CBD-Downtown,54076348498ed3ee81d32983,Scarritt Arcade,1361,819 Walnut St,39.103793,-94.581555,4bf58dd8d48988d130941735,Building
2,CBD-Downtown,5ad817f3c666667613ddcb70,Broadway Arcade,1449,1000 Broadway Blvd Ste 200,39.102272,-94.588670,4bf58dd8d48988d1e1931735,Arcade
3,CBD-Downtown,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1120,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
4,Columbus Park,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1437,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza


In [20]:
kc_venues = kc_venues[kc_venues != 'Null']
kc_venues = kc_venues[kc_venues != '[]']
kc_venues = kc_venues.dropna()
print(len(kc_venues))
kc_venues = kc_venues.reset_index()
kc_venues = kc_venues.drop(['index'], axis = 1)
kc_venues.head()

73


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,CBD-Downtown,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1130,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
1,CBD-Downtown,54076348498ed3ee81d32983,Scarritt Arcade,1361,819 Walnut St,39.103793,-94.581555,4bf58dd8d48988d130941735,Building
2,CBD-Downtown,5ad817f3c666667613ddcb70,Broadway Arcade,1449,1000 Broadway Blvd Ste 200,39.102272,-94.588670,4bf58dd8d48988d1e1931735,Arcade
3,CBD-Downtown,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1120,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
4,Columbus Park,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,1437,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza


### Sorting by Venue and Distance

In [21]:
kc_sorted = kc_venues.sort_values(['Venue_ID','Venue_Distance'], ascending = True)
kc_sorted.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
40,Hanover Place,4da3ab570f578cfa81a8be18,The Arcade,138,3632 Baltimore Ave,39.060734,-94.587339,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
49,Valentine,4da3ab570f578cfa81a8be18,The Arcade,526,3632 Baltimore Ave,39.060734,-94.587339,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
44,Old Westport,4da3ab570f578cfa81a8be18,The Arcade,839,3632 Baltimore Ave,39.060734,-94.587339,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
51,Westport,4da3ab570f578cfa81a8be18,The Arcade,839,3632 Baltimore Ave,39.060734,-94.587339,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)
65,Westport,4da3ab570f578cfa81a8be18,The Arcade,839,3632 Baltimore Ave,39.060734,-94.587339,4d954b06a243a5684965b473,Residential Building (Apartment / Condo)


In [22]:
kc_sorted['Category_Name'].unique()

array(['Residential Building (Apartment / Condo)', 'Event Space',
       'Miscellaneous Shop', 'Arcade', 'Shopping Plaza', 'Theme Park',
       'Building'], dtype=object)

In [23]:
kc_sorted = kc_sorted[kc_sorted['Category_Name'] != 'Residential Building (Apartment / Condo)']
kc_sorted['Category_Name'].unique()

array(['Event Space', 'Miscellaneous Shop', 'Arcade', 'Shopping Plaza',
       'Theme Park', 'Building'], dtype=object)

In [25]:
kc_sorted.to_csv('{}_sorted.csv'.format(search_query))

### Obtaining Neighborhood for Each Arcade

In [26]:
kc_sorted = pd.read_csv('{}_sorted.csv'.format(search_query))

In [27]:
kc_sorted = kc_sorted.drop('Unnamed: 0',axis =1 )

In [28]:
print(len(kc_sorted))
kc_sorted

52


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space
1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
2,Walnut Grove,4e68c94aae60186280a26fe8,Arcade Shoe Repair,371,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
3,Quality Hill,4e68c94aae60186280a26fe8,Arcade Shoe Repair,738,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
4,CBD-Downtown,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1120,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
5,Crossroads,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1284,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
6,River Market,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1302,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
7,Columbus Park,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1394,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
8,Parkview,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1510,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
9,West Side,4e68c94aae60186280a26fe8,Arcade Shoe Repair,1782,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop


In [29]:
count = -1
list_to_drop = list()
for ID in kc_sorted['Venue_ID']:
    count = count +1
    try:
        if kc_sorted['Venue_ID'][count] == kc_sorted['Venue_ID'][count-1] and kc_sorted['Venue_Distance'][count]>=kc_sorted['Venue_Distance'][count-1]:
            list_to_drop.append(count)
    except:
        continue
list_to_drop

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [30]:
kc_final = kc_sorted.drop(index = list_to_drop)

In [31]:
print(len(kc_final))
kc_final

7


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space
1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
13,Chaumiere,4fbefafae4b02c8542b9389a,Arcade Xbox,1305,No Address,39.162169,-94.546515,4bf58dd8d48988d1e1931735,Arcade
15,Library District,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,204,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
27,Winnetonka,51b790b2498e9e3b028cfaef,Joe Cool Arcade,1834,No Address,39.173578,-94.489309,4bf58dd8d48988d182941735,Theme Park
29,Walnut Grove,54076348498ed3ee81d32983,Scarritt Arcade,128,819 Walnut St,39.103793,-94.581555,4bf58dd8d48988d130941735,Building
41,Quality Hill,5ad817f3c666667613ddcb70,Broadway Arcade,152,1000 Broadway Blvd Ste 200,39.102272,-94.588670,4bf58dd8d48988d1e1931735,Arcade


In [33]:
kc_final.to_csv('{}_unique.csv'.format(search_query))

### Pulling the Ratings

In [34]:
search_query

'Arcade'

In [35]:
kc_final = pd.read_csv('{}_unique.csv'.format(search_query))
kc_final.head()

,Unnamed: 0,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space
1,1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
2,13,Chaumiere,4fbefafae4b02c8542b9389a,Arcade Xbox,1305,No Address,39.162169,-94.546515,4bf58dd8d48988d1e1931735,Arcade
3,15,Library District,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,204,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
4,27,Winnetonka,51b790b2498e9e3b028cfaef,Joe Cool Arcade,1834,No Address,39.173578,-94.489309,4bf58dd8d48988d182941735,Theme Park


In [37]:
kc_final = kc_final.drop('Unnamed: 0', axis = 1)
kc_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space
1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop
2,Chaumiere,4fbefafae4b02c8542b9389a,Arcade Xbox,1305,No Address,39.162169,-94.546515,4bf58dd8d48988d1e1931735,Arcade
3,Library District,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,204,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza
4,Winnetonka,51b790b2498e9e3b028cfaef,Joe Cool Arcade,1834,No Address,39.173578,-94.489309,4bf58dd8d48988d182941735,Theme Park


In [38]:
count = -1
rating_list = list()
for ident in kc_final['Venue_ID']:
    count = count +1
    venue_id = kc_final['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated']

In [39]:
kc_final = kc_final.assign(Venue_Rating = rating_list)

In [40]:
kc_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Coachlight Square,4e513b38c65bb313ba9ebab0,Sean's Arcade And Mini Bar,933,No Address,39.018198,-94.478832,4bf58dd8d48988d171941735,Event Space,Not Rated
1,Library District,4e68c94aae60186280a26fe8,Arcade Shoe Repair,221,No Address,39.101542,-94.581791,4bf58dd8d48988d1ff941735,Miscellaneous Shop,Not Rated
2,Chaumiere,4fbefafae4b02c8542b9389a,Arcade Xbox,1305,No Address,39.162169,-94.546515,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Library District,512bf4f7e4b0933b4ab63b1c,Commerce Arcade,204,1000 Walnut St,39.101503,-94.582445,5744ccdfe4b0c0459246b4dc,Shopping Plaza,Not Rated
4,Winnetonka,51b790b2498e9e3b028cfaef,Joe Cool Arcade,1834,No Address,39.173578,-94.489309,4bf58dd8d48988d182941735,Theme Park,Not Rated


In [41]:
kc_final.to_csv('{}_final_clean.csv'.format(search_query))